

# Indian start-up ecosystem


Your team is trying to venture into the Indian start-up ecosystem. As the data expert of the team, you are to investigate the ecosystem and propose the best course of action.
Analyze funding received by start-ups in India from 2018 to 2021.
Separate data for each year of funding will be provided.
• In these datasets, you'll find the start-ups' details, the funding amounts received, and the investors' information


#  Step 1

Importing the all the needed liberaries and the data ffor the project 

In [ ]:
# Neccesary liberaries
import pyodbc
from dotenv import dotenv_values
import pandas as pd
import numpy as np
from scipy import stats
import statistics as stat
import warnings
import os
 
warnings.filterwarnings('ignore')

In [15]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')
 
 
# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

In [16]:
# code to connect the sever, database, username and passward
conn_str =f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [18]:
# conection to the server
connection = pyodbc.connect(conn_str)


In [19]:
# import the 2020 and 2021 SQL file from the server
query1 = "Select * from LP1_startup_funding2020"
data_2020 = pd.read_sql(query1, connection)
 
query2 = "Select * from LP1_startup_funding2021"
data_2021 = pd.read_sql(query2, connection)

In [20]:
# 2019 CSV data
data_2019 = pd.read_csv('startup_funding2019.csv')

In [21]:
# 2018 CSV data
data_2018 = pd.read_csv('startup_funding2018.csv')

In [22]:
# to add year column to all the imported data
data_2018['Year']='2018'
data_2019['Year']='2019'
data_2020['Year']='2020'
data_2021['Year']='2021'

#  Step 2

cleaning and concating the two CSV files 2018 and 2019

In [ ]:
# Drop the Colunms that we may not need for the project

df2019.drop(['Founded', 'Founders', 'Investor'], axis=1, inplace=True)

In [23]:
# Select specific columns and rename them

df2018 = df2018.rename(columns={'Company Name': 'Company_Brand', 'Round/Series': 'Stage', 'Industry': 'Sector', 'Amount': 'Amount', 'About Company': 'BIO', 'Location':'HeadQuarter'})
df2019 = df2019.rename(columns={'Company/Brand': 'Company_Brand', 'Sector': 'Sector', 'Stage': 'Stage', 'Amount($)': 'Amount', 'What it does': 'BIO', 'HeadQuarter':'HeadQuarter'})

Hamornizing the stage column in 2018 & 2019

In [24]:
# df2018 Dataframe 

df2018.replace(to_replace=['Seed round', 'Seed funding', 'Early seed', 'Seed fund', 'Seed Investment', 'Seed Round', 'Seed+'], value='Seed', inplace=True)
df2018.replace(to_replace=['Angel Round'], value='Angel', inplace=True)
df2018.replace(to_replace=['Venture - Series Unknown'], value='Undisclosed', inplace=True)
df2018.replace(to_replace=['Debt Financing', 'Post-IPO Debt'], value='Debt', inplace=True)
df2018.replace(to_replace=['Private Equity', 'Post-IPO Equity'], value='Equity', inplace=True)
df2018.replace(to_replace=['Corporate Round', 'Grant', 'Secondary Market', 'Non-equity Assistance', 'Funding Round'], value='Other', inplace=True)
df2018.replace(to_replace=['Pre-Series B', 'Pre-series B'], value='Pre-Seed', inplace=True)

# df2019 Dataframe

df2019.replace(to_replace=['Seed funding', 'Seed fund', 'Seed round'], value='Seed', inplace=True)
df2019.replace(to_replace=['Pre series A', 'Pre-series A'], value='Pre-Seed', inplace=True)
df2019.replace(to_replace=['Series B+'], value='Series B', inplace=True)
df2019.replace(to_replace=['Post series A', 'Fresh funding'], value='Other', inplace=True)


# Remove non-standard entries

df2018 = df2018[df2018['Stage'].str.startswith('https') == False]

In [12]:
# Cleaning the sector column
# create a new column in 2019, 2020 and 2021 with tech and another that has sector that do not have tech
data_2019['Tech'] = data_2019['Sector'].apply(lambda x: x if 'Tech' in str(x) or 'tech' in str(x) or 'AI' in str(x) or 'IoT' in str(x) else None)
data_2019['without_tech'] = data_2019['Sector'].apply(lambda x: x if 'Tech' not in str(x) and 'tech' not in str(x) and 'AI' not in str(x) and 'IoT' not in str(x) else None)
#2020
data_2020['Tech'] = data_2020['Sector'].apply(lambda x: x if 'Tech' in str(x) or 'tech' in str(x) or 'AI' in str(x) or 'IoT' in str(x) else None)
data_2020['without_tech'] = data_2020['Sector'].apply(lambda x: x if 'Tech' not in str(x) and 'tech' not in str(x) and 'AI' not in str(x) and 'IoT' not in str(x) else None)
#2021
data_2021['Tech'] = data_2021['Sector'].apply (lambda x:x if 'Tech' in str(x) or 'tech' in str(x) or'AI' in str(x) or 'IoT' in str(x) else None)
data_2021['without_tech']=data_2021['Sector'].apply ( lambda x:x if 'Tech'not in str(x) and 'tech' not in str(x) and 'AI' not in str(x) and 'IoT' not in str(x) else None)


<b>Harmonising the HeadQuater/Location column entries in 2018&2019 data.<b>

In [ ]:
# code to sparete the headquarters in 2018
data_2018_new=data_2018['HeadQuarter'].str.split(pat=',', n=1, expand=True)
data_2018['Head']=data_2018_new[0]

In [ ]:
# drop headquarter column
data_2018.drop(['HeadQuarter'], axis=1, inplace=True)
# rename the head column
data_2018.rename(columns={'Head':'HeadQuarter'}, inplace=True)

In [ ]:
# data cleaning of headquarter in 2018
data_2018.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data_2018.replace(to_replace=['Delhi','New Delhi'], value='Delhi', inplace=True)
data_2018.replace(to_replace=['Cochin'], value='Kochi', inplace=True)
data_2018.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data_2018.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data_2018.replace(to_replace=['Uttar Pradesh', 'Andhra Pradesh'], value='Andhra Pradesh', inplace=True)
data_2018.replace(to_replace=['Trivandrum', 'India'], value='Trivandrum', inplace=True)


# data cleaning of headquarter in 2019
data_2019.replace(to_replace=['Delhi,New Delhi'], value='Delhi', inplace=True)

<b>CHECK FOR DUPLICATE ENTRIES IN THE NEW DATASET df2018 & df2019<b>

In [ ]:
df2018.duplicated().value_counts()

In [ ]:
df2018.drop_duplicates(keep='first', inplace=True)

In [ ]:
df2019.duplicated().value_counts()

# Step 3 

Merging 2018 & 2019 and Checking for currency

In [ ]:
# Merge the two csv files 

df18_19 = pd.concat([df2018, df2019], ignore_index=True)

In [ ]:
# Create a new column 'currency' based on currency symbols

df18_19['currency'] = df18_19['Amount'].apply(lambda x: 'INR' if '₹' in x else ('USD' if '$' in x else 'USD'))

In [ ]:
# Create columns 'amount_inr' and 'amount_usd'

df18_19['amount_inr'] = df18_19.apply(lambda row: row['Amount'] if row['currency'] == 'INR' else 0, axis=1)
df18_19['amount_usd'] = df18_19.apply(lambda row: row['Amount'] if row['currency'] == 'USD' else 0, axis=1)

In [ ]:
# Remove the currency sysmbols and the commas 

df18_19['amount_inr'] = df18_19['amount_inr'].str.replace('₹', '').str.replace('$', '').str.replace(',', '')
df18_19['amount_usd'] = df18_19['amount_usd'].str.replace('₹', '').str.replace('$', '').str.replace(',', '')

In [ ]:
# Replace 'Undisclosed' values with NaN

df18_19['amount_inr'] = df18_19['amount_inr'].replace('Undisclosed', np.nan, regex=True)
df18_19['amount_usd'] = df18_19['amount_usd'].replace('Undisclosed', np.nan, regex=True)

In [ ]:
# Convert the object values into numeric values for calculation

df18_19['amount_inr'] = pd.to_numeric(df18_19['amount_inr'], errors='coerce')
df18_19['amount_usd'] = pd.to_numeric(df18_19['amount_usd'], errors='coerce')

In [ ]:
# Create a new column 'Updated_Amount' by filling missing values in 'amount_inr' with 'amount_usd'
df18_19['Updated_Amount'] = df18_19['amount_inr'].fillna(df18_19['amount_usd'])

In [ ]:
df18_19.duplicated().value_counts()

# Step 4

Cleaning and concating the two SQL file 2021 & 2018

In [ ]:
# Renaming the Column

data2020 = data2020.rename(columns={'Company_Brand': 'Company_Brand', 'Sector': 'Sector', 'Amount': 'Amount', 'What_it_does': 'BIO', 'Location':'HeadQuarter'})
data2021 = data2021.rename(columns={'Company_Brand': 'Company_Brand', 'Sector': 'Sector', 'Amount': 'Amount', 'What_it_does': 'BIO', 'HeadQuarter':'HeadQuarter'})

<b> Drop some columns that might not be needed for the analysis <b>

In [ ]:
data2020.drop(['Founded', 'Founders', 'Investor'], axis=1, inplace=True)
data2021.drop(['Founded', 'Founders', 'Investor'], axis=1, inplace=True)
data2020.drop(columns=['column10'], inplace=True)

<b>Harmonising the Stage column entries in 2020&2021 data.<b>

In [ ]:
# data2020 DataFrame 

data2020.replace(to_replace=['Seed Round', 'Seed round', 'Seed funding', 'Seed A', 'Seed Funding', 'Seed Investment','Pre seed Round', 'Seed funding' 'Seed A' 'Pre-seed', 'Pre seed round', 'Pre-seed Round', 'Pre-Seed'], value='Seed', inplace=True)
data2020.replace(to_replace=['Series A1', 'Series A-1'], value='Series A', inplace=True)
data2020.replace(to_replace=['Series B2'], value='Series B', inplace=True)
data2020.replace(to_replace=['Pre series B', 'Pre-Series B', 'Pre series C', 'Pre-Series B' 'Pre series C','Pre series A1', 'Pre-series', 'Pre series A', 'Pre-series A', 'Pre- series A', 'Pre Series A', 'Pre-series A1', 'Pre-series C', 'Pre-series B'], value='Pre-Series', inplace=True)
data2020.replace(to_replace=['Series E', 'Series E2', 'Series F'], value='Series E and Beyond', inplace=True)
data2020.replace(to_replace=['Series C, D', 'Series D', 'Series D1'], value='Series D', inplace=True)
data2020.replace(to_replace=['Bridge Round', 'Edge', 'Bridge'], value='Bridge', inplace=True)
data2020.replace(to_replace=['Angel Round'], value='Angel', inplace=True)




# data2021 DataFrame

data2021.replace(to_replace=['Seed Round', 'Seed round', 'Seed funding', 'Seed A', 'Seed+', 'Seed Funding', 'Seed Investment','Pre seed Round', 'Early seed', 'Seed funding' 'Seed A' 'Pre-seed', 'Pre seed round', 'Pre-seed', 'Pre-seed Round', 'Pre-Seed'], value='Seed', inplace=True)
data2021.replace(to_replace=['Series A1', 'Seies A', 'Series A2', 'Series A+', 'Series A-1'], value='Series A', inplace=True)
data2021.replace(to_replace=['Series B2', 'Series B3'], value='Series B', inplace=True)
data2021.replace(to_replace=['Pre series B', 'Pre-Series B', 'Pre series C', 'Pre-Series B' 'Pre series C','Pre series A1', 'Pre-series', 'Pre series A', 'Pre-series A', 'Pre- series A', 'Pre Series A', 'Pre-series A1', 'Pre-series C', 'Pre-series B'], value='Pre-Series', inplace=True)
data2021.replace(to_replace=['Series E', 'Series I', 'Series F1', 'Series H', 'Series G', 'Series F2', 'Series E2', 'Series F'], value='Series E and Beyond', inplace=True)
data2021.replace(to_replace=['Series C, D', 'Series D', 'Series D1'], value='Series D', inplace=True)
data2021.replace(to_replace=['Bridge Round', 'Edge', 'Bridge'], value='Bridge', inplace=True)
data2021.replace(to_replace=['Angel Round', '$300000', '$1200000', '$6000000', '$1000000'], value='Angel', inplace=True)

<b>Harmonising the Sector column entries in 2020&2021 data.<b>

In [ ]:
# data cleaning of Sector in 2020

data2020['Tech'] = data2020['Sector'].apply(lambda x: x if 'Tech' in str(x) or 'tech' in str(x) or 'AI' in str(x) or 'IoT' in str(x) else None)
data2020['Non_tech'] = data2020['Sector'].apply(lambda x: x if 'Tech' not in str(x) and 'tech' not in str(x)
                                                       and 'AI' not in str(x) and 'IoT' not in str(x) else None)

In [ ]:
# data cleaning of Sector in 2021

data2021['Tech'] = data2021['Sector'].apply(lambda x: x if 'Tech' in str(x) or 'tech' in str(x) or 'AI' in str(x) or 'IoT' in str(x) else None)
data2021['Non_tech'] = data2021['Sector'].apply(lambda x: x if 'Tech' not in str(x) and 'tech' not in str(x)
                                                       and 'AI' not in str(x) and 'IoT' not in str(x) else None)

<b>Harmonising the HeadQuarter column entries in 2020&2021 data.<b>

In [ ]:
#
#  cleaning of HeadQuater 2020
data_2020.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data_2020.replace(to_replace=['Delhi,New Delhi'], value='Delhi', inplace=True)
data_2020.replace(to_replace=['Ahmadabad'], value='Ahmedabad', inplace=True)
data_2020.replace(to_replace=['Kochi'], value='Cochin', inplace=True)
data_2020.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data_2020.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data_2020.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data_2020.replace(to_replace=['Small Towns, Andhra Pradesh','Uttar Pradesh'], value='Andhra Pradesh', inplace=True)
data_2020.replace(to_replace=['Hyderebad'], value='Hyderabad', inplace=True)
data_2020.replace(to_replace=['Gurugram\t#REF!'], value='Gurugram', inplace=True)
data_2020.replace(to_replace=['Orissia'], value='Orissa', inplace=True)
data_2020.replace(to_replace=['Samstipur','Samastipur, Bihar','Samsitpur'], value='Samastipur', inplace=True)
data_2020.replace(to_replace=['The Nilgiris'], value='Nilgiris', inplace=True)
data_2020.replace(to_replace=['Dhindsara, Haryana','Dhingsara, Haryana'], value='Dhingsara', inplace=True)
data_2020.replace(to_replace=['Tirunelveli, Tamilnadu'], value='Tirunelveli', inplace=True)
data_2020.replace(to_replace=['Mylapore'], value='Chennai', inplace=True)
data_2020.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data_2020.replace(to_replace=['Trivandrum, Kerala, India'], value='Trivandrum', inplace=True)

In [28]:
#  Renaming some of the headquarters the are not in India to outside india in data 2020
data_2020.replace(to_replace=['Frisco', 'Newcastle Upon Tyne, Newcastle upon Tyne, United Kingdom','Irvine, California, United States','San Francisco Bay Area, West Coast, Western US','Texas, United States','California','New York, United States','San Francisco, California, United States','San Francisco, United States','San Ramon, California','Paris, Ile-de-France, France','Plano, Texas, United States','Sydney','San Francisco Bay Area, Silicon Valley, West Coast','Bangaldesh','London, England, United Kingdom','Sydney, New South Wales, Australia','Milano, Lombardia, Italy','Palmwoods, Queensland, Australia', 'France''Irvine, California, United States','Newcastle Upon Tyne, Newcastle upon Tyne United Kingdom','Shanghai, China','Jiaxing, Zhejiang, China','San Franciscao','San Francisco','New York'], value = 'Outside India', inplace=True)

In [32]:
# filling of missing headquarters in 2020
data_2020['HeadQuarter'].fillna(method='ffill', inplace=True)

In [15]:
#  cleaning of HeadQuater 2021
data_2021.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data_2021.replace(to_replace=['Delhi,New Delhi'], value='Delhi', inplace=True)
data_2021.replace(to_replace=['Ahmadabad'], value='Ahmedabad', inplace=True)
data_2021.replace(to_replace=['Kochi'], value='Cochin', inplace=True)
data_2021.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data_2021.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data_2021.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data_2021.replace(to_replace=['Small Towns, Andhra Pradesh'], value='Andhra Pradesh', inplace=True)
data_2021.replace(to_replace=['Hyderebad'], value='Hyderabad', inplace=True)
data_2021.replace(to_replace=['Gurugram\t#REF!'], value='Gurugram', inplace=True)
data_2021.replace(to_replace=['The Nilgiris'], value='Nilgiris', inplace=True)
data_2021.replace(to_replace=['Dhindsara, Haryana','Dhingsara, Haryana'], value='Dhingsara', inplace=True)
data_2021.replace(to_replace=['Tirunelveli, Tamilnadu'], value='Tirunelveli', inplace=True)
data_2021.replace(to_replace=['Mylapore'], value='Chennai', inplace=True)
data_2021.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data_2021.replace(to_replace=['Trivandrum, Kerala, India'], value='Trivandrum', inplace=True)

In [ ]:
#  Renaming some of the headquarters the are not in India to outside india in data 2020
data2021.replace(to_replace=['London', 'New York', ' Mountain View', 'CA ', 'Beijing', '' ], value='Outside_India', inplace=True)
data2021.replace(to_replace=['Online Media\t#REF!', 'Pharmaceuticals\t#REF!', 'Food & Beverages', 
                             'Computer Games', 'Small Towns', ' None', 'None', 'Information Technology & Services'], value='Undisclosed', inplace=True)
data2021.replace(to_replace=['Gurugram\t#REF!' ], value='Gurugrama', inplace=True)

In [25]:
# filling of missing headquarters in 2020
data_2021['HeadQuarter'].fillna(method='ffill', inplace=True)

In [26]:
# filling of missing headquarters in 2020
data_2019['HeadQuarter'].fillna(method='ffill', inplace=True)

In [19]:
# code to sparete the headquarters in 2018
data_2018_new=data_2018['HeadQuarter'].str.split(pat=',', n=1, expand=True)
data_2018['Head']=data_2018_new[0]


In [20]:
# drop headquarter column
data_2018.drop(['HeadQuarter'], axis=1, inplace=True)


In [21]:
# drop headquarter column
data_2018.drop(['HeadQuarter'], axis=1, inplace=True)
# rename the head column
data_2018.rename(columns={'Head':'HeadQuarter'}, inplace=True)

In [22]:
# data cleaning of headquarter in 2018
data_2018.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data_2018.replace(to_replace=['Delhi','New Delhi'], value='Delhi', inplace=True)
data_2018.replace(to_replace=['Cochin'], value='Kochi', inplace=True)
data_2018.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data_2018.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data_2018.replace(to_replace=['Uttar Pradesh', 'Andhra Pradesh'], value='Andhra Pradesh', inplace=True)
data_2018.replace(to_replace=['Trivandrum', 'India'], value='Trivandrum', inplace=True)

In [27]:
data_2020.isna().sum()

Company_Brand      0
HeadQuarter        0
Sector            13
Bio                0
Amount           254
Stage            464
Year               0
Tech             533
without_tech     535
dtype: int64

In [15]:
# filling of missing headquarters in 2020
data_2020['HeadQuarter'].fillna(method='ffill', inplace=True)

In [15]:
data_2020.isna().sum()

Company_Brand      0
HeadQuarter        0
Sector            13
Bio                0
Amount           254
Stage            464
Year               0
dtype: int64

In [16]:
data_2021.head(2)

,Company_Brand,HeadQuarter,Sector,Bio,Amount,Stage,Year
0,Unbox Robotics,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"$1,200,000",Pre-series A,2021
1,upGrad,Mumbai,EdTech,UpGrad is an online higher education platform.,"$120,000,000",None,2021


In [17]:
data_2021.isna().sum()


Company_Brand      0
HeadQuarter        1
Sector             0
Bio                0
Amount             3
Stage            428
Year               0
dtype: int64

In [30]:
count_2020 = data_2020['HeadQuarter'].value_counts()

In [31]:
count_2020.tail(50)

Andheri                                                     2
San Francisco, California, United States                    2
London, England, United Kingdom                             2
Goa                                                         2
San Francisco Bay Area, Silicon Valley, West Coast          2
Frisco, Texas, United States                                1
Seoul                                                       1
Bangkok                                                     1
New York                                                    1
Riyadh                                                      1
Belgaum                                                     1
Seattle                                                     1
Berlin                                                      1
Tangerang, Jawa Barat, Indonesia                            1
Kanpur                                                      1
Odisha                                                      1
Warangal

In [19]:
# filling of missing values in 2021
data_2021['HeadQuarter'].fillna(method='ffill', inplace=True)

In [19]:
data_2021.isna().sum()

Company_Brand      0
HeadQuarter        0
Sector             0
Bio                0
Amount             3
Stage            428
Year               0
dtype: int64

In [20]:
data20_21 = pd.concat([data_2020, data_2021], ignore_index=True)

In [21]:
data20_21

,Company_Brand,HeadQuarter,Sector,Bio,Amount,Stage,Year
0,Aqgromalin,Chennai,AgriTech,Cultivating Ideas for Profit,200000.0,None,2020
1,Krayonnz,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,100000.0,Pre-seed,2020
2,PadCare Labs,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,NaN,Pre-seed,2020
3,NCOME,New Delhi,Escrow,Escrow-as-a-service platform,400000.0,None,2020
4,Gramophone,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,340000.0,None,2020
...,...,...,...,...,...,...,...
2259,Gigforce,Gurugram,Staffing & Recruiting,A gig/on-demand staffing company.,$3000000,Pre-series A,2021
2260,Vahdam,New Delhi,Food & Beverages,VAHDAM is among the world’s first vertically i...,$20000000,Series D,2021
2261,Leap Finance,Bangalore,Financial Services,International education loans for high potenti...,$55000000,Series C,2021
2262,CollegeDekho,Gurugram,EdTech,"Collegedekho.com is Student’s Partner, Friend ...",$26000000,Series B,2021


In [22]:
data20_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2264 entries, 0 to 2263
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  2264 non-null   object
 1   HeadQuarter    2264 non-null   object
 2   Sector         2251 non-null   object
 3   Bio            2264 non-null   object
 4   Amount         2007 non-null   object
 5   Stage          1372 non-null   object
 6   Year           2264 non-null   object
dtypes: object(7)
memory usage: 123.9+ KB


In [82]:
data20_21['HeadQuarter'].unique()


array(['Chennai', 'Bangalore', 'Pune', 'New Delhi', 'Indore', 'Hyderabad',
       'Gurgaon', 'Belgaum', 'Noida', 'Mumbai', 'Andheri', 'Jaipur',
       'Ahmedabad', 'Kolkata', 'Tirunelveli', 'Thane', 'Singapore',
       'Gurugram', 'Gujarat', 'Haryana', 'Kerala', 'Jodhpur',
       'Jaipur, Rajastan', 'Delhi', 'Frisco, Texas, United States',
       'California', 'Dhingsara', 'New York, United States', 'Patna',
       'San Francisco, California, United States',
       'San Francisco, United States', 'San Ramon, California',
       'Paris, Ile-de-France, France', 'Plano, Texas, United States',
       'Sydney', 'San Francisco Bay Area, Silicon Valley, West Coast',
       'Bangaldesh', 'London, England, United Kingdom',
       'Sydney, New South Wales, Australia', 'Milano, Lombardia, Italy',
       'Palmwoods, Queensland, Australia', 'France',
       'San Francisco Bay Area, West Coast, Western US', 'Trivandrum',
       'Cochin', 'Samastipur', 'Irvine, California, United States',
       'Tum

In [52]:
# cleaning of HeadQuater 2020and 2021
data20_21.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data20_21.replace(to_replace=['Delhi,New Delhi'], value='Delhi', inplace=True)
data20_21.replace(to_replace=['Ahmadabad'], value='Ahmedabad', inplace=True)
data20_21.replace(to_replace=['Kochi'], value='Cochin', inplace=True)
data20_21.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data20_21.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data20_21.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data20_21.replace(to_replace=['Small Towns, Andhra Pradesh'], value='Andhra Pradesh', inplace=True)
data20_21.replace(to_replace=['Hyderebad'], value='Hyderabad', inplace=True)
data20_21.replace(to_replace=['Gurugram\t#REF!'], value='Gurugram', inplace=True)
data20_21.replace(to_replace=['Orissia'], value='Orissa', inplace=True)
data20_21.replace(to_replace=['Samstipur','Samastipur, Bihar','Samsitpur'], value='Samastipur', inplace=True)
data20_21.replace(to_replace=['The Nilgiris'], value='Nilgiris', inplace=True)
data20_21.replace(to_replace=['Dhindsara, Haryana','Dhingsara, Haryana'], value='Dhingsara', inplace=True)
data20_21.replace(to_replace=['Tirunelveli, Tamilnadu'], value='Tirunelveli', inplace=True)
data20_21.replace(to_replace=['Mylapore'], value='Chennai', inplace=True)
data20_21.replace(to_replace=['Rajastan'], value='Rajasthan', inplace=True)
data20_21.replace(to_replace=['Trivandrum, Kerala, India'], value='Trivandrum', inplace=True)


Where are most startups located in India?
Top 5 cities in India emerging as the startup hub of India
Bengaluru
Bengaluru, also known as the “Silicon Valley of India,” has a strong ecosystem for technology startups and is home to many successful companies such as Flipkart, Ola, and InMobi.1 Feb 2023

In [26]:
data20_21['Updated_Amount'] = data20_21['Amount']

In [27]:
data20_21['with_$'] = data20_21['Updated_Amount'].apply(lambda x: x if '$' in str(x) else None)
data20_21['without_$'] = data20_21['Updated_Amount'].apply(lambda x: x if '$' not in str(x) else None)

#remove the currency symbols and commas
data20_21['with_$'] = data20_21['with_$'].str.replace('$', '')


#Replace undisclose amount with NAN
data20_21['with_$'] = data20_21['with_$'].replace('Undisclosed', np.nan, regex=True)


# Convert the object values into numeric values for calculation

data20_21['with_$'] = pd.to_numeric(data20_21['with_$'], errors='coerce')
data20_21['without_$'] = pd.to_numeric(data20_21['without_$'], errors='coerce')

# Merge the two Colunms into a new colunm 

data20_21['Updated_Amount'] = data20_21['without_$'].fillna(data20_21['with_$'])


In [28]:
data20_21.drop(['with_$','without_$'], axis=1,inplace=True)

In [29]:
data20_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2264 entries, 0 to 2263
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company_Brand   2264 non-null   object 
 1   HeadQuarter     2264 non-null   object 
 2   Sector          2251 non-null   object 
 3   Bio             2264 non-null   object 
 4   Amount          2007 non-null   object 
 5   Stage           1372 non-null   object 
 6   Year            2264 non-null   object 
 7   Updated_Amount  1514 non-null   float64
dtypes: float64(1), object(7)
memory usage: 141.6+ KB


In [31]:
data20_21['Sector'].unique

<bound method Series.unique of 0                    AgriTech
1                      EdTech
2          Hygiene management
3                      Escrow
4                    AgriTech
                ...          
2259    Staffing & Recruiting
2260         Food & Beverages
2261       Financial Services
2262                   EdTech
2263       Financial Services
Name: Sector, Length: 2264, dtype: object>

Hypothesis 
Hypothesis:  What the company dose depends on the amount of funding secured
Null Hypothesis: What the company dose not have infulence  on the amount of funding secured 

  Hypothesis 
Hypothesis:  The amount of start up funding has infulence on the kind of investors 

Null Hypothesis: The amount of start up funding has no significance infulence on the investors 

In [39]:
data_2019.shape

(89, 7)

In [83]:
data_2019['HeadQuarter'].fillna(method='bfill', inplace=True)

In [114]:
data_2019['HeadQuarter'].unique()

array(['Mumbai', 'Chennai', 'Telangana', 'Pune', 'Bangalore', 'Noida',
       'Delhi', 'Ahmedabad', 'Gurugram', 'Haryana', 'Chandigarh',
       'Jaipur', 'New Delhi', 'Surat', 'Uttar pradesh', 'Hyderabad',
       'Rajasthan'], dtype=object)

In [ ]:
data_2019.info()

In [56]:
data_2019.isna().sum()

Company_Brand     0
HeadQuarter       0
Sector            5
BIO               0
Amount            0
Stage            46
Year              0
dtype: int64

In [97]:
data_2019.head(2)

,Company_Brand,HeadQuarter,Sector,BIO,Amount,Stage,Amount,Year
0,Bombay Shaving,Mumbai,Ecommerce,Provides a range of male grooming products,"$6,300,000",NaN,"$6,300,000",2019
1,Ruangguru,Mumbai,Edtech,A learning platform that provides topic-based ...,"$150,000,000",Series C,"$150,000,000",2019


In [113]:
# data cleaning of headquarter in 2019
data20_21.replace(to_replace=['Delhi,New Delhi'], value='Delhi', inplace=True)

In [100]:

data_2018 

,Company_Brand,Sector,Stage,Amount,HeadQuarter,BIO,Year
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f...",2018
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...,2018
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India,2018
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...,2018
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...,2018
...,...,...,...,...,...,...,...
521,Udaan,"B2B, Business Development, Internet, Marketplace",Series C,225000000,"Bangalore, Karnataka, India","Udaan is a B2B trade platform, designed specif...",2018
522,Happyeasygo Group,"Tourism, Travel",Series A,—,"Haryana, Haryana, India",HappyEasyGo is an online travel domain.,2018
523,Mombay,"Food and Beverage, Food Delivery, Internet",Seed,7500,"Mumbai, Maharashtra, India",Mombay is a unique opportunity for housewives ...,2018
524,Droni Tech,Information Technology,Seed,"₹35,000,000","Mumbai, Maharashtra, India",Droni Tech manufacture UAVs and develop softwa...,2018


In [110]:
data_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  526 non-null    object
 1   Sector         526 non-null    object
 2   Stage          526 non-null    object
 3   Amount         526 non-null    object
 4   HeadQuarter    526 non-null    object
 5   BIO            526 non-null    object
 6   Year           526 non-null    object
dtypes: object(7)
memory usage: 28.9+ KB


In [124]:
data_2018_new=data_2018['HeadQuarter'].str.split(pat=',', n=1, expand=True)
data_2018['Head']=data_2018_new[0]
data_2018


,Company_Brand,Sector,Stage,Amount,HeadQuarter,BIO,Year,Head
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f...",2018,Bangalore
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...,2018,Mumbai
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India,2018,Gurgaon
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...,2018,Noida
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...,2018,Hyderabad
...,...,...,...,...,...,...,...,...
521,Udaan,"B2B, Business Development, Internet, Marketplace",Series C,225000000,"Bangalore, Karnataka, India","Udaan is a B2B trade platform, designed specif...",2018,Bangalore
522,Happyeasygo Group,"Tourism, Travel",Series A,—,"Haryana, Haryana, India",HappyEasyGo is an online travel domain.,2018,Haryana
523,Mombay,"Food and Beverage, Food Delivery, Internet",Seed,7500,"Mumbai, Maharashtra, India",Mombay is a unique opportunity for housewives ...,2018,Mumbai
524,Droni Tech,Information Technology,Seed,"₹35,000,000","Mumbai, Maharashtra, India",Droni Tech manufacture UAVs and develop softwa...,2018,Mumbai


In [148]:

data_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  526 non-null    object
 1   Sector         526 non-null    object
 2   Stage          526 non-null    object
 3   Amount         526 non-null    object
 4   BIO            526 non-null    object
 5   Year           526 non-null    object
 6   HeadQuarter    526 non-null    object
dtypes: object(7)
memory usage: 28.9+ KB


In [147]:
data_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  89 non-null     object
 1   HeadQuarter    89 non-null     object
 2   Sector         84 non-null     object
 3   BIO            89 non-null     object
 4   Amount         89 non-null     object
 5   Stage          43 non-null     object
 6   Year           89 non-null     object
dtypes: object(7)
memory usage: 5.0+ KB


In [125]:
data_2018['Head'].unique()


array(['Bangalore', 'Mumbai', 'Gurgaon', 'Noida', 'Hyderabad',
       'Bengaluru', 'Kalkaji', 'Delhi', 'India', 'Hubli', 'New Delhi',
       'Chennai', 'Mohali', 'Kolkata', 'Pune', 'Jodhpur', 'Kanpur',
       'Ahmedabad', 'Azadpur', 'Haryana', 'Cochin', 'Faridabad', 'Jaipur',
       'Kota', 'Anand', 'Bangalore City', 'Belgaum', 'Thane', 'Margão',
       'Indore', 'Alwar', 'Kannur', 'Trivandrum', 'Ernakulam',
       'Kormangala', 'Uttar Pradesh', 'Andheri', 'Mylapore', 'Ghaziabad',
       'Kochi', 'Powai', 'Guntur', 'Kalpakkam', 'Bhopal', 'Coimbatore',
       'Worli', 'Alleppey', 'Chandigarh', 'Guindy', 'Lucknow'],
      dtype=object)

In [126]:
# data cleaning of headquarter in 2018
data_2018.replace(to_replace=['Bangalore City','Bangalore'], value='Bangalore', inplace=True)
data_2018.replace(to_replace=['Delhi','New Delhi'], value='Delhi', inplace=True)
data_2018.replace(to_replace=['Cochin'], value='Kochi', inplace=True)
data_2018.replace(to_replace=['Kormangala'], value='Koramangala', inplace=True)
data_2018.replace(to_replace=['Powai','Worli'], value='Mumbai', inplace=True)
data_2018.replace(to_replace=['Uttar Pradesh', 'Andhra Pradesh'], value='Andhra Pradesh', inplace=True)
data_2018.replace(to_replace=['Trivandrum', 'India'], value='Trivandrum', inplace=True)

In [138]:
data_2018_new=data_2018['HeadQuarter'].str.split(pat=',', n=1, expand=True)
data_2018['Head']=data_2018_new[0]
data_2018
data_2018.drop(['HeadQuarter'], axis=1, inplace=True)
data_2018.rename(columns={'Head':'HeadQuarter'}, inplace=True)

In [141]:
data_2018.rename(columns={'Head':'HeadQuarter'}, inplace=True)


In [142]:


data_2018

,Company_Brand,Sector,Stage,Amount,BIO,Year,HeadQuarter
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"TheCollegeFever is a hub for fun, fiesta and f...",2018,Bangalore
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000",A startup which aggregates milk from dairy far...,2018,Mumbai
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000",Leading Online Loans Marketplace in India,2018,Gurgaon
3,PayMe India,"Financial Services, FinTech",Angel,2000000,PayMe India is an innovative FinTech organizat...,2018,Noida
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,Eunimart is a one stop solution for merchants ...,2018,Hyderabad
...,...,...,...,...,...,...,...
521,Udaan,"B2B, Business Development, Internet, Marketplace",Series C,225000000,"Udaan is a B2B trade platform, designed specif...",2018,Bangalore
522,Happyeasygo Group,"Tourism, Travel",Series A,—,HappyEasyGo is an online travel domain.,2018,Haryana
523,Mombay,"Food and Beverage, Food Delivery, Internet",Seed,7500,Mombay is a unique opportunity for housewives ...,2018,Mumbai
524,Droni Tech,Information Technology,Seed,"₹35,000,000",Droni Tech manufacture UAVs and develop softwa...,2018,Mumbai


In [143]:

data18_19 =pd.concat([data_2018,data_2019], axis=1, ignore_index=True)

In [144]:
data18_19

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"TheCollegeFever is a hub for fun, fiesta and f...",2018,Bangalore,Bombay Shaving,Mumbai,Ecommerce,Provides a range of male grooming products,"$6,300,000",NaN,2019
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000",A startup which aggregates milk from dairy far...,2018,Mumbai,Ruangguru,Mumbai,Edtech,A learning platform that provides topic-based ...,"$150,000,000",Series C,2019
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000",Leading Online Loans Marketplace in India,2018,Gurgaon,Eduisfun,Mumbai,Edtech,It aims to make learning fun via games.,"$28,000,000",Fresh funding,2019
3,PayMe India,"Financial Services, FinTech",Angel,2000000,PayMe India is an innovative FinTech organizat...,2018,Noida,HomeLane,Chennai,Interior design,Provides interior designing solutions,"$30,000,000",Series D,2019
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,Eunimart is a one stop solution for merchants ...,2018,Hyderabad,Nu Genes,Telangana,AgriTech,"It is a seed company engaged in production, pr...","$6,000,000",NaN,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Udaan,"B2B, Business Development, Internet, Marketplace",Series C,225000000,"Udaan is a B2B trade platform, designed specif...",2018,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,Happyeasygo Group,"Tourism, Travel",Series A,—,HappyEasyGo is an online travel domain.,2018,Haryana,NaN,NaN,NaN,NaN,NaN,NaN,NaN
523,Mombay,"Food and Beverage, Food Delivery, Internet",Seed,7500,Mombay is a unique opportunity for housewives ...,2018,Mumbai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524,Droni Tech,Information Technology,Seed,"₹35,000,000",Droni Tech manufacture UAVs and develop softwa...,2018,Mumbai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
